# Distributed data parallel MaskRCNN training with TensorFlow2 and SMDataParallel

SMDataParallel is a new capability in Amazon SageMaker to train deep learning models faster and cheaper. SMDataParallel is a distributed data parallel training framework for TensorFlow, PyTorch, and MXNet.

This notebook example shows how to use SMDataParallel with TensorFlow(version 2.3.1) on [Amazon SageMaker](https://aws.amazon.com/sagemaker/) to train a MaskRCNN model on [COCO 2017 dataset](https://cocodataset.org/#home) using [Amazon FSx for Lustre file-system](https://aws.amazon.com/fsx/lustre/) as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Create Amazon FSx Lustre file-system and import data into the file-system from S3
3. Build Docker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
4. Configure data input channels for SageMaker
5. Configure hyper-prarameters
6. Define training metrics
7. Define training job, set distribution strategy to SMDataParallel and start training

**NOTE:**  With large traning dataset, we recommend using (Amazon FSx)[https://aws.amazon.com/fsx/] as the input filesystem for the SageMaker training job. FSx file input to SageMaker significantly cuts down training start up time on SageMaker because it avoids downloading the training data each time you start the training job (as done with S3 input for SageMaker training job) and provides good data read throughput.


**NOTE:** This example requires SageMaker Python SDK v2.X.

## Amazon SageMaker Initialization

Initialize the notebook instance. Get the aws region, sagemaker execution role.

The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with the appropriate full IAM role arn string(s). As described above, since we will be using FSx, please make sure to attach `FSx Access` permission to this IAM role.

In [1]:
%%time
! python3 -m pip install --upgrade sagemaker




     |████████████████████████████████| 396 kB 10.2 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.23.4.post0-py2.py3-none-any.whl size=559728 sha256=64cbccea6853eb344235a19b009133e017b056512ca62f7568649c0dd2df3078
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f4/bf/4d/b0f85f925dd2e323e2e159e33204ae7340fd55bf4745797a3d
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.0
    Uninstalling smdebug-rulesconfig-1.0.0:
      Successfully uninstalled smdebug-rulesconfig-1.0.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.19.0
    Uninstalling sagemaker-2.19.0:
      Successfully uninstalled sagemaker-2.19.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python3 -m pip install --upgrade pip' command.
CPU times: user 115 ms, sys: 36.6 ms, total: 152 ms
Wall time: 14 s


In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

SageMaker Execution Role:arn:aws:iam::230755935769:role/SageMakerExecutionRoleMLOps
AWS account:230755935769
AWS region:us-west-2


## Prepare SageMaker Training Images

1. SageMaker by default use the latest [Amazon Deep Learning Container Images (DLC)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) TensorFlow training image. In this step, we use it as a base image and install additional dependencies required for training MaskRCNN model.
2. In the Github repository https://github.com/HerringForks/DeepLearningExamples.git we have made TensorFlow-SMDataParallel MaskRCNN training script available for your use. We will be installing the same on the training image.

### Build and Push Docker Image to ECR

Run the below command build the docker image and push it to ECR.

In [2]:
image = "tf2-mask-rcnn-smdataparallel-sagemaker"  # Example: tf2-mask-rcnn-smdataparallel-sagemaker
tag = "latest"   # Example: latest

In [3]:
!pygmentize ./Dockerfile

ARG region

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04

RUN 	pip --no-cache-dir --no-cache install \
        Cython \
        matplotlib \
        opencv-python-headless \
        mpi4py \
        Pillow \
        pytest \
        pyyaml

RUN 	cd /root && \
	git clone https://github.com/pybind/pybind11 && \
	cd pybind11 && \
	cmake . && \
	make -j96 install && \
	pip install .

RUN 	pip --no-cache-dir --no-cache install \
    	'git+https://github.com/NVIDIA/cocoapi#egg=pycocotools&subdirectory=PythonAPI' && \
	pip --no-cache-dir --no-cache install \
    	'git+https://github.com/NVIDIA/dllogger'


In [4]:
!pygmentize ./build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# set region

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"

if [ "$#" -eq 3 ]; then
    region=$1
    image=$2
    tag=$3
else
    echo "usage: $0 <aws-region> $1 <image-repo> $2 <image-tag>"
    exit 1
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "creating ECR repository : ${fullname} "
    aw

In [5]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  285.2kB
Step 1/5 : ARG region
Step 2/5 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04
2.3.1-gpu-py37-cu110-ubuntu18.04: Pulling from tensorflow-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
9f1ec0f3: Pulling fs layer 
556b2329: Waiting fs layer 
9f1ec0f3: Waiting fs layer 
3d9de4de: Waiting fs layer 
3ad2a50a: Pulling fs layer 
c51c9781: Waiting fs layer 
badfd864: Pulling fs layer 
3ad2a50a: Waiting fs layer 
035444b2: Pulling fs layer 
a1a16d74: Wai

51c9781: Extracting  1.094GB/2.914GBB[49ADownloading  47.25MB/345.8MBe2aa5793: Pull complete .33kB/38.33kBExtracting  507.5MB/591.3MBDigest: sha256:9266326cd7d9ab373d3a17ee703ca67770dad1d8341e025e376003f1a2d432cd
Status: Downloaded newer image for 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04
 ---> 32790263d5eb
Step 3/5 : RUN 	pip --no-cache-dir --no-cache install         Cython         matplotlib         opencv-python-headless         mpi4py         Pillow         pytest         pyyaml
 ---> Running in 6d6798c3aa47


Removing intermediate container 6d6798c3aa47
 ---> c0b53859ba55
Step 4/5 : RUN 	cd /root && 	git clone https://github.com/pybind/pybind11 && 	cd pybind11 && 	cmake . && 	make -j96 install && 	pip install .
 ---> Running in 8c8d9c172a66
Cloning into 'pybind11'...
-- The CXX compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- pybind11 v2.6.2 dev1
CMake Warning (dev) at CMakeLists.txt:57 (message):
  You are building in-place.  If that is not what you intended to do, you can
  clean the source directory with:

  rm -r CMakeCache.txt CMakeFiles/ cmake_uninstall.cmake pybind11Config.cmake
  pybind11ConfigVersion.cmake tests/CMakeFiles/

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CMake 3.18.2
-- Found PythonInterp: /usr/local/bin/python3.7 (found version "3.7.7") 
--

    Preparing wheel metadata: finished with status 'done'
  Created wheel for pybind11: filename=pybind11-2.6.2.dev1-py2.py3-none-any.whl size=190538 sha256=03145db1eefa7dc13e0fd9fa6bbd3e325c687b3394b9a194426968dde5d52a18
  Stored in directory: /tmp/pip-ephem-wheel-cache-pmnt69dg/wheels/2a/e8/f8/e5daba934e808130ebb389b2832ca810f17864b69c253e5058
Successfully built pybind11
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.6.1
    Uninstalling pybind11-2.6.1:
      Successfully uninstalled pybind11-2.6.1
Removing intermediate container 8c8d9c172a66
 ---> 169ca864d021
Step 5/5 : RUN 	pip --no-cache-dir --no-cache install     	'git+https://github.com/NVIDIA/cocoapi#egg=pycocotools&subdirectory=PythonAPI' && 	pip --no-cache-dir --no-cache install     	'git+https://github.com/NVIDIA/dllogger'
 ---> Running in be4bf5a22e5e
  Cloning https://github.com/NVIDIA/cocoapi to /tmp/pip-install-ulidw8nk/pycocotools_478ca8d924f344bc94ec470c65b701ad
  Created wheel for pycoco

## Preparing FSx Input for SageMaker

1. Download and prepare your training dataset on S3.
2. Follow the steps listed here to create a FSx linked with your S3 bucket with training data - https://docs.aws.amazon.com/fsx/latest/LustreGuide/create-fs-linked-data-repo.html. Make sure to add an endpoint to your VPC allowing S3 access.
3. Follow the steps listed here to configure your SageMaker training job to use FSx https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/

### Important Caveats

1. You need use the same `subnet` and `vpc` and `security group` used with FSx when launching the SageMaker notebook instance. The same configurations will be used by your SageMaker training job.
2. Make sure you set appropriate inbound/output rules in the `security group`. Specically, opening up these ports is necessary for SageMaker to access the FSx filesystem in the training job. https://docs.aws.amazon.com/fsx/latest/LustreGuide/limit-access-security-groups.html
3. Make sure `SageMaker IAM Role` used to launch this SageMaker training job has access to `AmazonFSx`.

## SageMaker TensorFlow Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`.

### Training script

In the Github repository https://github.com/HerringForks/DeepLearningExamples.git we have made reference TensorFlow-SMDataParallel MaskRCNN training script available for your use. Clone the repository.

In [6]:
# Clone herring (smdataparallel) forks repository for reference implementation of H
!rm -rf DeepLearningExamples
!git clone --recursive https://github.com/HerringForks/DeepLearningExamples.git

Cloning into 'DeepLearningExamples'...
remote: Enumerating objects: 23025, done.
remote: Total 23025 (delta 0), reused 0 (delta 0), pack-reused 23025
Receiving objects: 100% (23025/23025), 57.78 MiB | 17.92 MiB/s, done.
Resolving deltas: 100% (17769/17769), done.


In [7]:
from sagemaker.tensorflow import TensorFlow

In [44]:
instance_type = "ml.p3.16xlarge" # Other supported instance type: ml.p3.16xlarge
instance_count = 2 # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}" # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
username = 'AWS'
subnets = ['subnet-b3ec00f9'] # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = ['sg-fddcc4ae'] # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = 'tf2-smdataparallel-mrcnn-fsx-3' # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id='fs-02362f5e7db00888b' # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'


In [45]:
SM_DATA_ROOT = '/opt/ml/input/data/train'

hyperparameters={
    "mode": "train",
    "checkpoint": '/'.join([SM_DATA_ROOT, 'model/resnet/resnet-nhwc-2018-02-07/model.ckpt-112603']), 
    "eval_samples": 5000,
    "init_learning_rate": 0.04, 
    "learning_rate_steps": "3750,5000", 
    "model_dir": "/opt/ml/model/", 
    "num_steps_per_eval": 462,
    "total_steps": 500,
    "train_batch_size": 4,
    "eval_batch_size": 8,
    "training_file_pattern": '/'.join([SM_DATA_ROOT, 'train2017']), 
    "validation_file_pattern": '/'.join([SM_DATA_ROOT, 'val2017']), 
    "val_json_file": '/'.join([SM_DATA_ROOT, 'annotations/instances_val2017.json']),    
    "amp": '',
    "use_batched_nms": '',
    "xla": '',
    "nouse_custom_box_proposals_op": '',
    "seed": 987
    }

In [46]:
estimator = TensorFlow(entry_point='DeepLearningExamples/TensorFlow2/Segmentation/MaskRCNN/mask_rcnn_sm.py',
                        role=role,
                        image_uri=docker_image,
                        source_dir='.',
                        framework_version='2.3.1',
                        py_version='py3',
                        instance_count=instance_count,
                        instance_type=instance_type,
                        sagemaker_session=sagemaker_session,
#                         subnets=subnets,
                        hyperparameters=hyperparameters,
#                         security_group_ids=security_group_ids,
                        debugger_hook_config=False,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                                 }
                                          }
                                      }
                      )

In [47]:
# Configure FSx Input for your SageMaker Training job
# change to s3 
# from sagemaker.inputs import FileSystemInput
# file_system_directory_path='/fsx' # NOTE: '/fsx/' will be the root mount path. Example: '/fsx/mask_rcnn/PyTorch'
# file_system_access_mode='rw'
# file_system_type='FSxLustre'
# train_fs = FileSystemInput(file_system_id=file_system_id,
#                                     file_system_type=file_system_type,
#                                     directory_path=file_system_directory_path,
#                                     file_system_access_mode=file_system_access_mode)

train_fs = sagemaker.inputs.TrainingInput('s3://coco2017-yianc-20210114/')
data_channels = {'train': train_fs}

In [48]:
# Submit SageMaker training job
estimator.fit(inputs=data_channels, job_name=job_name)

2021-01-16 00:11:31 Starting - Starting the training job...
2021-01-16 00:11:54 Starting - Launching requested ML instancesProfilerReport-1610755850: InProgress
............
2021-01-16 00:13:56 Starting - Preparing the instances for training......
2021-01-16 00:14:56 Downloading - Downloading input data............................................................................................................................................................
2021-01-16 00:40:56 Training - Downloading the training image...........................2021-01-16 00:45:32.153363: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-01-16 00:45:32.160632: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-01-16 00:45:32.524279: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully op


2021-01-16 00:46:00 Training - Training image download completed. Training in progress.2021-01-16 00:46:00,392 sagemaker-training-toolkit INFO     Process[es]: [psutil.Process(pid=180, name='orted', status='sleeping', started='00:45:59')]
2021-01-16 00:46:00,392 sagemaker-training-toolkit INFO     Orted process found [psutil.Process(pid=180, name='orted', status='sleeping', started='00:45:59')]
2021-01-16 00:46:00,393 sagemaker-training-toolkit INFO     Waiting for orted process [psutil.Process(pid=180, name='orted', status='sleeping', started='00:45:59')]
[1,8]<stdout>:algo-2:209:209 [0] NCCL INFO Bootstrap : Using [0]eth0:10.0.203.160<0>
[1,8]<stdout>:algo-2:209:209 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
[1,8]<stdout>:
[1,8]<stdout>:algo-2:209:209 [0] misc/ibvwrap.cc:63 NCCL WARN Failed to open libibverbs.so[.1]
[1,8]<stdout>:algo-2:209:209 [0] NCCL INFO NET/Socket : Using [0]eth0:10.0.203.160<0>
[1,8]<stdout>:algo-2:209:209 [0] NC

[1,4]<stdout>:algo-1:200:200 [4] NCCL INFO Channel 07 : 4[1b0] -> 0[170] via P2P/IPC
[1,11]<stdout>:algo-2:208:208 [3] NCCL INFO Channel 07 : 3[1a0] -> 0[170] via P2P/IPC
[1,0]<stdout>:algo-1:196:196 [0] NCCL INFO Channel 07 : 0[170] -> 3[1a0] via P2P/IPC
[1,14]<stdout>:algo-2:214:214 [6] NCCL INFO Channel 07 : 6[1d0] -> 5[1c0] via P2P/IPC
[1,13]<stdout>:algo-2:210:210 [5] NCCL INFO Channel 07 : 5[1c0] -> 1[180] via P2P/IPC
[1,15]<stdout>:algo-2:213:213 [7] NCCL INFO Channel 07 : 7[1e0] -> 6[1d0] via P2P/IPC
[1,1]<stdout>:algo-1:197:197 [1] NCCL INFO Channel 07 : 1[180] -> 5[1c0] via P2P/IPC
[1,3]<stdout>:algo-1:195:195 [3] NCCL INFO Channel 07 : 3[1a0] -> 2[190] via P2P/IPC
[1,2]<stdout>:algo-1:198:198 [2] NCCL INFO Channel 07 : 2[190] -> 1[180] via P2P/IPC
[1,5]<stdout>:algo-1:201:201 [5] NCCL INFO Channel 07 : 5[1c0] -> 6[1d0] via P2P/IPC
[1,6]<stdout>:algo-1:204:204 [6] NCCL INFO Channel 07 : 6[1d0] -> 7[1e0] via P2P/IPC
[1,7]<stdout>:algo-1:205:205 [7] NCCL INFO Channel 07 : 7[1e0

[1,0]<stdout>:[MaskRCNN] INFO    : SageMaker Distributed Data Parallel successfully initialized ...
[1,0]<stdout>:[MaskRCNN] INFO    : [train] AMP is activated - Experiment Feature
[1,0]<stdout>:[MaskRCNN] INFO    : [train] XLA is activated - Experiment Feature
[1,0]<stdout>:[MaskRCNN] INFO    : Create CheckpointSaverHook.
[1,0]<stdout>:[MaskRCNN] INFO    : Using Dataset Sharding with SageMaker Distributed Data Parallel
[1,0]<stdout>:[MaskRCNN] INFO    : [ROI OPs] Using Batched NMS... Scope: multilevel_propose_rois/level_2/
[1,0]<stdout>:[MaskRCNN] INFO    : [ROI OPs] Using Batched NMS... Scope: multilevel_propose_rois/level_3/
[1,0]<stdout>:[MaskRCNN] INFO    : [ROI OPs] Using Batched NMS... Scope: multilevel_propose_rois/level_4/
[1,0]<stdout>:[MaskRCNN] INFO    : [ROI OPs] Using Batched NMS... Scope: multilevel_propose_rois/level_5/
[1,0]<stdout>:[MaskRCNN] INFO    : [ROI OPs] Using Batched NMS... Scope: multilevel_propose_rois/level_6/
[1,0]<stdout>:[MaskRCNN] INFO    : [Training C

[1,11]<stdout>:    session.run(self._initializer)
[1,11]<stdout>:  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 958, in run
[1,11]<stdout>:    run_metadata_ptr)
[1,11]<stdout>:  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1181, in _run
[1,11]<stdout>:    feed_dict_tensor, options, run_metadata)
[1,11]<stdout>:  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1359, in _do_run
[1,11]<stdout>:    run_metadata)
[1,11]<stdout>:  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1384, in _do_call
[1,11]<stdout>:    raise type(e)(node_def, op, message)
[1,11]<stdout>:tensorflow.python.framework.errors_impl.InvalidArgumentError: assertion failed: [No files matched pattern: /opt/ml/input/data/train/train2017/train*.tfrecord]
[1,11]<stdout>:#011 [[node list_files/assert_not_empty/Assert (defined at opt/ml/code/DeepLearningExamples/Te

2021-01-16 00:46:39,274 sagemaker-training-toolkit INFO     Orted process exited

2021-01-16 00:47:13 Failed - Training job failed
ProfilerReport-1610755850: Stopping
2021-01-16 00:47:09,299 sagemaker-training-toolkit INFO     MPI process finished.
2021-01-16 00:47:09,299 sagemaker_tensorflow_container.training WARNING  No model artifact is saved under path /opt/ml/model. Your training job will not save any model files to S3.
For details of how to construct your training script see:
https://sagemaker.readthedocs.io/en/stable/using_tf.html#adapting-your-local-tensorflow-script
2021-01-16 00:47:09,299 sagemaker-training-toolkit INFO     Reporting training SUCCESS


UnexpectedStatusException: Error for Training job tf2-smdataparallel-mrcnn-fsx-3: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "mpirun --host algo-1:8,algo-2:8 -np 16 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca plm_rsh_num_concurrent 2 -x NCCL_SOCKET_IFNAME=eth0 -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_HOMOGENEOUS=1 -x FI_PROVIDER=sockets -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/usr/local/lib/python3.7/site-packages/gethostname.cpython-37m-x86_64-linux-gnu.so -x SMDATAPARALLEL_SERVER_ADDR=algo-1 -x SMDATAPARALLEL_SERVER_PORT=7592 -x SAGEMAKER_INSTANCE_TYPE=ml.p3.16xlarge smddprun /usr/local/bin/python3.7 -m mpi4py DeepLearningExamples/TensorFlow2/Segmentation/MaskRCNN/mask_rcnn_sm.py --amp  --checkpoint /opt/ml/input/data/train/model/resnet/resnet-nhwc-2018-02-07/model.ckpt-112603 --eval_batch_size 8 --

# Additional Resources

If you are a new user of Amazon SageMaker, you may find the following helpful to understand how SageMaker uses Docker to train custom models.
* To learn more about using Amazon SageMaker with your own training image, see [Use Your Own Training Algorithms
](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html).

* To learn more about using Docker to train your own models with Amazon SageMaker, see [Example Notebooks: Use Your Own Algorithm or Model](https://docs.aws.amazon.com/sagemaker/latest/dg/adv-bring-own-examples.html).
* To see other examples of distributed training using Amazon SageMaker and TensorFlow, see [Distributed TensorFlow training using Amazon SageMaker
](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/distributed_tensorflow_mask_rcnn).